# Лабораторная 2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Задача: сделать оптимизацию select-запросов
Перед выполнением загрузите файл database.zip и ctrl+F9

Код создания базы данных (для понимания, что в ней находится)

```
CREATE TABLE employees ( -- сотрудники компании
    emp_no      INT             NOT NULL,
    birth_date  DATE            NOT NULL,
    first_name  VARCHAR(14)     NOT NULL,
    last_name   VARCHAR(16)     NOT NULL,
    gender      CHAR(2)         NOT NULL,    
    hire_date   DATE            NOT NULL,
    PRIMARY KEY (emp_no)
);

CREATE TABLE departments ( -- отделения
    dept_no     CHAR(4)         NOT NULL,
    dept_name   VARCHAR(40)     NOT NULL,
    PRIMARY KEY (dept_no)
);

CREATE TABLE dept_manager ( -- менеджеры отделений
   dept_no      CHAR(4)         NOT NULL,
   emp_no       INT             NOT NULL,
   from_date    DATE            NOT NULL,
   to_date      DATE            NOT NULL,
   FOREIGN KEY (emp_no)  REFERENCES employees (emp_no)    ON DELETE CASCADE,
   FOREIGN KEY (dept_no) REFERENCES departments (dept_no) ON DELETE CASCADE,
   PRIMARY KEY (emp_no,dept_no)
); 

CREATE TABLE dept_emp (  -- связь сотрудников и отделений, даты работы
    emp_no      INT             NOT NULL,
    dept_no     CHAR(4)         NOT NULL,
    from_date   DATE            NOT NULL,
    to_date     DATE            NOT NULL,
    FOREIGN KEY (emp_no)  REFERENCES employees   (emp_no)  ON DELETE CASCADE,
    FOREIGN KEY (dept_no) REFERENCES departments (dept_no) ON DELETE CASCADE,
    PRIMARY KEY (emp_no,dept_no)
);

CREATE TABLE titles ( -- должности
    emp_no      INT             NOT NULL,
    title       VARCHAR(50)     NOT NULL,
    from_date   DATE            NOT NULL,
    to_date     DATE,
    FOREIGN KEY (emp_no) REFERENCES employees (emp_no) ON DELETE CASCADE,
    PRIMARY KEY (emp_no,title, from_date)
); 

CREATE TABLE salaries ( -- зарплаты
    emp_no      INT             NOT NULL,
    salary      INT             NOT NULL,
    from_date   DATE            NOT NULL,
    to_date     DATE            NOT NULL,
    FOREIGN KEY (emp_no) REFERENCES employees (emp_no) ON DELETE CASCADE,
    PRIMARY KEY (emp_no, from_date)
); 
```

In [2]:
import zipfile

with zipfile.ZipFile('database.zip') as zf:
    zf.extractall()

In [3]:
%load_ext sql
%sql sqlite:///database.db

'Connected: @database.db'

1. Выбрать имена сотрудников с датой рождения до 1 января 1980 года или с должностью помощника инженера 

In [4]:
%%time
%%sql
SELECT distinct first_name, last_name FROM employees e
INNER JOIN salaries s        --присоединение 1
    ON s.emp_no = e.emp_no 
    WHERE (
        (e.birth_date < '1980-01-01') --условие 1
        OR e.emp_no IN (              --условие 2
        select emp_no from titles t   --вложенный запрос 1
            where t.title like 'Assistant%'
            order by from_date)       --странная сортировка 1
    )
LIMIT 10

 * sqlite:///database.db
Done.
CPU times: user 6.96 ms, sys: 0 ns, total: 6.96 ms
Wall time: 7.83 ms


first_name,last_name
Georgi,Facello
Bezalel,Simmel
Parto,Bamford
Chirstian,Koblick
Kyoichi,Maliniak
Anneke,Preusig
Tzvetan,Zielinski
Saniya,Kalloufi
Sumant,Peac
Duangkaew,Piveteau


In [5]:
%%time
%%sql
SELECT DISTINCT first_name  
                ,last_name 
FROM employees e
INNER JOIN salaries USING(emp_no)
INNER JOIN titles   USING(emp_no)
WHERE (birth_date   <   '1980-01-01') OR 
      (title      like  'Assistant%')
LIMIT 10

 * sqlite:///database.db
Done.
CPU times: user 5.28 ms, sys: 0 ns, total: 5.28 ms
Wall time: 5.33 ms


first_name,last_name
Georgi,Facello
Bezalel,Simmel
Parto,Bamford
Chirstian,Koblick
Kyoichi,Maliniak
Anneke,Preusig
Tzvetan,Zielinski
Saniya,Kalloufi
Sumant,Peac
Duangkaew,Piveteau


2. Выбрать сотрудников, в настоящее время работающие в отделах, где менеджеры мужчины

In [6]:
%%time
%%sql
SELECT first_name, last_name FROM employees e 
INNER JOIN dept_emp d           
    ON d.emp_no = e.emp_no
    WHERE d.dept_no in (        
    SELECT d.dept_no FROM employees e, dept_manager dm, departments d
        WHERE e.emp_no = dm.emp_no AND dm.dept_no = d.dept_no 
        AND dm.to_date = (SELECT MAX(to_date) from dept_manager)      
        AND e.gender = 'M'                                            
        GROUP BY d.dept_no
    )
    AND d.to_date = (SELECT MAX(to_date) from dept_emp)      --что-то надо придумать с этим.
LIMIT 10

 * sqlite:///database.db
Done.
CPU times: user 34.9 ms, sys: 4.8 ms, total: 39.7 ms
Wall time: 41.2 ms


first_name,last_name
Bezalel,Simmel
Parto,Bamford
Chirstian,Koblick
Sumant,Peac
Duangkaew,Piveteau
Kazuhito,Cappelletti
Cristinel,Bouloucos
Kazuhide,Peha
Mayuko,Warwick
Suzette,Pettey


In [13]:
%%time
%%sql
SELECT first_name
      ,last_name FROM employees e 
INNER JOIN dept_emp d USING(emp_no)
    WHERE d.dept_no in (        
    SELECT d.dept_no FROM employees e
    INNER JOIN dept_manager dm USING(emp_no)
    INNER JOIN departments d   USING(dept_no)
        WHERE dm.to_date = '9999-01-01'       
        AND e.gender = 'M'
    )
    AND d.to_date = '9999-01-01'    --using with or const           
LIMIT 10                            --использование констант плохо, но оно работает

 * sqlite:///database.db
Done.
CPU times: user 6.49 ms, sys: 0 ns, total: 6.49 ms
Wall time: 7.35 ms


first_name,last_name
Bezalel,Simmel
Parto,Bamford
Chirstian,Koblick
Sumant,Peac
Duangkaew,Piveteau
Kazuhito,Cappelletti
Cristinel,Bouloucos
Kazuhide,Peha
Mayuko,Warwick
Suzette,Pettey


In [12]:
%sql SELECT max(to_date) from dept_emp

 * sqlite:///database.db
Done.


max(to_date)
9999-01-01
